In [ ]:
#@title Install necessary libraries
!pip install langchain_community pandas huggingface_hub chromadb==0.4.14 langchain-huggingface langchain_groq lark openai gradio python-dotenv qrcode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Define Constant Variables

In [ ]:
#@title Get API Keys from environment variables
def get_key(key,required=True):
    try:
        from google.colab.userdata import get as getenv
    except ImportError:
        from os import getenv
        import dotenv
        dotenv.load_dotenv()

    value = getenv(key)

    if not value:
        value = input(f"Please enter the value for {key}:")
        if value == "" and required:
            raise ValueError(f"Value for {key} is required")

    return value

In [8]:
#@title Set tokens, keys, and default directories
# Tokens and API Keys
huggingface_token = get_key("HUGGINGFACE_KEY")
groq_api_key = get_key("GROQ_KEY")

# Directories
DATA_DIR = './data'
persist_directory = DATA_DIR+"/chroma_db"
embedding_cache_directory = DATA_DIR+"/embedding_cache"
USER_AGENT="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 YaBrowser/24.4.0.0 Safari/537.36"

In [9]:
from huggingface_hub import login
login(token=huggingface_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load the necessary documents

In this step we will import the documents from the datasets prepared by the scraper. Including:
* List of university courses (csv)
* Description and necessary information of each course (json, csv)

Also:
* The system message for the chatbot (txt)
* General university information (web)

In [32]:
import pandas as pd

course_list_df = pd.read_json(DATA_DIR + '/courses_list.json')
course_desc_df = pd.read_json(DATA_DIR + '/courses_description_legacy.json')


def get_course_unit_content(course_unit):
    content = f"""
    The course with code {course_unit['Code']}, named "{course_unit['Name']}" (English name: "{course_unit['English name']}"), is coordinated by {course_unit['Coordinator']}. It is part of the {course_unit['Course type']} program in the {course_unit['Department']} department under the {course_unit['School']} school. The course is conducted in {course_unit['Language']} and is located in {course_unit['Locations']}. This course was activated in the year {course_unit['Activation year']}. For more information, visit the course homepage: {course_unit['Course Homepage']}.
    """
    return content.strip()

# def rename_columns(df, target_column_name, new_name):
#     return df.rename(columns={target_column_name: new_name})

def load_courses_document_df(course_desc_df):
    course_desc_df['content'] = course_desc_df.apply(lambda x: get_course_unit_content(x), axis=1)
    metadata = ["Code", "Name", "English name", "Coordinator", "content"]
    return course_desc_df[metadata]

df = load_courses_document_df(course_desc_df)
df

,Code,Name,English name,Coordinator,content
0,N70,ARCHEOLOGIA E STORIA DELL'ARTE,ARCHAEOLOGY AND HISTORY OF ART,DI MEGLIO ROSALBA,"The course with code N70, named ""ARCHEOLOGIA E..."
1,P14,"ARCHEOLOGIA, STORIA DELLE ARTI E SCIENZE DEL P...","ARCHEOLOGY, HISTORY OF ARTS AND SCIENCE OF ART...",CICALA LUIGI,"The course with code P14, named ""ARCHEOLOGIA, ..."
2,P53,ARCHITECTURE AND HERITAGE,ARCHITECTURE AND HERITAGE,PANE ANDREA,"The course with code P53, named ""ARCHITECTURE ..."
3,D06,ARCHITETTURA,Not indicated,CERRETA MARIA,"The course with code D06, named ""ARCHITETTURA""..."
4,N14,ARCHITETTURA,ARCHITECTURE,CERRETA MARIA,"The course with code N14, named ""ARCHITETTURA""..."
...,...,...,...,...,...
182,N10,TECNOLOGIE ALIMENTARI,FOOD TECHNOLOGY,FERRANTI PASQUALE,"The course with code N10, named ""TECNOLOGIE AL..."
183,N72,TECNOLOGIE DELLE PRODUZIONI ANIMALI,Technologies in animal productions,CALABRO' SERENA,"The course with code N72, named ""TECNOLOGIE DE..."
184,P22,TOSSICOLOGIA CHIMICA E AMBIENTALE,CHEMISTRY AND ENVIROMENTAL TOXICOLOGY,SORRENTINO RAFFAELLA,"The course with code P22, named ""TOSSICOLOGIA ..."
185,P55,TRANSPORTATION ENGINEERING AND MOBILITY,TRANSPORTATION ENGINEERING AND MOBILITY,BIFULCO GENNARO NICOLA,"The course with code P55, named ""TRANSPORTATIO..."


In [52]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain_community.document_loaders.dataframe import DataFrameLoader
import bs4

# Define system message and metadata documents
system_message = TextLoader(DATA_DIR + '/system_message.txt')

metadata = TextLoader(DATA_DIR + '/description_metadata.txt')

# Let's load help desk and other services
webDocs = WebBaseLoader(
    ['https://www.international.unina.it/welcome-desk/'],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_="post-2026 page type-page status-publish ast-article-single"
        )
    ),
)

# Let's load the course list and details documents
# course_list_df = DataFrameLoader(load_courses_document_df(course_desc_df), 'content')
course_desc_df = DataFrameLoader(df, 'content')

# Merge all the available documents
merged_loader = MergedDataLoader([system_message, webDocs, course_desc_df, metadata])
documents = merged_loader.load()

print(f'There are {len(documents)} pages in total')

There are 190 pages in total


In [51]:
#@title Retreive a random document
import random

random_index = random.randint(0, len(documents))
document_content = documents[random_index].page_content
document_metadata = documents[random_index].metadata
print(document_content,"\n---\n",document_metadata)

The course with code N86, named "CORSO DI LAUREA IN INFORMATICA" (English name: "1ST DEGREE IN COMPUTER SCIENCE"), is coordinated by DI MARTINO SERGIO. It is part of the Laurea program in the DIPARTIMENTO DI INGEGNERIA ELETTRICA E TECNOLOGIE DELL'INFORMAZIONE department under the SCUOLA POLITECNICA E DELLE SCIENZE DI BASE school. The course is conducted in Italian and is located in NAPOLI. This course was activated in the year 2008. For more information, visit the course homepage: https://www.corsi.unina.it/N86. 
---
 {'Code': 'N86', 'Name': 'CORSO DI LAUREA IN INFORMATICA', 'English name': '1ST DEGREE IN COMPUTER SCIENCE', 'Coordinator': 'DI MARTINO SERGIO'}


## Efficient Document Processing through Segmentation

By breaking down large documents into smaller, more manageable chunks, we can streamline our processing efforts and achieve greater efficiency. This process, known as text segmentation or splitting, involves dividing a massive document into distinct, bite-sized pieces. By doing so, we can analyze and process the text in a more detailed and structured manner, enabling us to extract valuable insights and information from the data.

While segmenting documents may potentially disrupt the original context or meaning of the text, we can employ various techniques to mitigate this issue. For instance, we can utilize recursive splitting methods or incorporate techniques that preserve the original context and meaning of the text.

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=60,
    separators=["\n\n", "\n", " ", ""]
)

texts = text_splitter.split_documents(documents)

print(f'We have created {len(texts)} chunks from {len(documents)} pages')

We have created 417 chunks from 190 pages


## Vector Representation Using Embedding Models

When converting text into numerical representations, we rely on various embedding models to transform the text into dense vectors that capture its semantic meaning. Popular options include BERT, RoBERTa, GPT-2, among others. In this example, we utilize the MiniLM model, a smaller variant of the BERT architecture. By applying an embedding model, we can convert text into numerical vectors that encode the semantic essence of the text. These vectors can be employed for various natural language processing tasks, such as:

* Text classification: categorizing text based on its content
* Clustering: grouping similar texts together
* Similarity search: identifying the most relevant texts based on their semantic meaning

The embedding model plays a crucial role in enabling these NLP applications by providing a compact and meaningful representation of the original text.


In [49]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Efficient Embedding Retrieval Using Caching

When processing new documents, our approach first checks the cache for pre-computed embeddings. If these embeddings are not readily available, the method employs an underlying embedder to generate the embeddings and stores them in the cache for future reference.

By leveraging caching, we can:

* Avoid recalculating embeddings each time, which can be computationally expensive
* Retrieve embeddings from memory, making our process faster and more efficient

To optimize our embedding-based workflow, let's set up a cache to store pre-computed embeddings. This will enable us to quickly retrieve relevant
information and reduce the computational overhead associated with re-calculating embeddings.


In [53]:
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings

store = LocalFileStore(embedding_cache_directory)
embedding_model = CacheBackedEmbeddings.from_bytes_store(embedding_model, store)

### Vector Database for Efficient Storage and Retrieval

Vector databases are designed to efficiently store and retrieve vectors, making them an essential component in applications such as information retrieval, recommendation systems, and clustering. These databases enable fast similarity searches by comparing query vectors with stored vectors, allowing us to quickly identify the most relevant matches.

In this notebook, we utilize the Chroma vector database, which is optimized for tasks involving similarity search and retrieval.

**Important Note**: Due to the relatively small size of our documents, it's more efficient to pass entire documents to the vector database at once, rather than splitting them. This approach simplifies the process and minimizes potential errors.

After the initial vectorization step, we load pre-computed vectors from cache on subsequent calls, ensuring a seamless and optimized experience for future interactions with the vector database.


In [54]:
import os
from langchain_community.vectorstores import Chroma


# Check if persist_directory exists or not
if os.path.exists(persist_directory):
    # Read from cache
    vectordb = Chroma(
        persist_directory= persist_directory ,
        embedding_function=embedding_model
    )
else:
    # Vectorize documents without splitting
    vectordb = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_directory
    )

Let's assess the similarity between the retrieved documents and the original query:


In [62]:
result = vectordb.similarity_search(
    'Who is the person in charge for autonomous vehicle engineering course?',
    #k=6
)

print(f"for query {len(result)} results found")
print(result[0].page_content)
print(result[0].metadata)

for query 4 results found
The course with code D18, named "AUTONOMOUS VEHICLE ENGINEERING" (English name: "AUTONOMOUS VEHICLE ENGINEERING"), is coordinated by PATALANO STANISLAO. It is part of the Laurea Magistrale program in the DIPARTIMENTO DI INGEGNERIA INDUSTRIALE department under the SCUOLA POLITECNICA E DELLE SCIENZE DI BASE school. The course is conducted in English and is located in NAPOLI. This course was activated in the year 2023. For more information, visit the course homepage: https://www.corsi.unina.it/D18.
{'Code': 'D18', 'Coordinator': 'PATALANO STANISLAO', 'English name': 'AUTONOMOUS VEHICLE ENGINEERING', 'Name': 'AUTONOMOUS VEHICLE ENGINEERING'}


In [65]:

result = vectordb.similarity_search(
    'Artificial Intelligence',
    # k=6
)
print(f"for query {len(result)} results found")
print(result[0].page_content)
print(result[0].metadata)

for query 4 results found
The course with code P38, named "INGEGNERIA DELL'AUTOMAZIONE E ROBOTICA" (English name: "AUTOMATION ENGINEERING AND ROBOTICS"), is coordinated by DE TOMMASI GIANMARIA. It is part of the Laurea Magistrale program in the DIPARTIMENTO DI INGEGNERIA ELETTRICA E TECNOLOGIE DELL'INFORMAZIONE department under the SCUOLA POLITECNICA E DELLE SCIENZE DI BASE school. The course is conducted in Italian and is located in NAPOLI. This course was activated in the year 2019. For more information, visit the course homepage: https://www.corsi.unina.it/P38.
{'Code': 'P38', 'Coordinator': 'DE TOMMASI GIANMARIA', 'English name': 'AUTOMATION ENGINEERING AND ROBOTICS', 'Name': "INGEGNERIA DELL'AUTOMAZIONE E ROBOTICA"}


### Utilizing the LLM with Groq API

To leverage the power of language models, let's load the LLaMA model using the Groq API. This will enable us to tap into the capabilities of this
pre-trained model without having to fine-tune it for our specific task. By utilizing the Groq API, we can streamline our workflow and optimize
performance.

In particular, I plan to utilize the `nvidia/Llama3-ChatQA-1.5-8B` model from Hugging Face's model hub. This model has been specifically designed for
chat-style question answering tasks and is expected to provide strong performance in this domain.


In [66]:
from langchain_groq import ChatGroq
chat_model = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

#### Evaluating Model Performance on Unseen Questions

Now, let's put the model to the test by asking it questions and assessing its ability to provide accurate answers. Since we're working with an unseen
dataset, this exercise will help us gauge the model's generalizability and ability to generalize beyond its training data.


In [70]:
qa_result = chat_model.invoke(
    "Who is the person in charge for autonomous vehicle engineering course in University of Naples Federico II?"
)
print(qa_result.content)

I've searched for the University of Naples Federico II's Autonomous Vehicle Engineering course, but I couldn't find a specific person in charge for the course. The university's website doesn't provide a single point of contact for the course.

However, I can suggest some alternatives to help you find the contact information:

1. **Contact the Department of Engineering**: You can try contacting the Department of Engineering at the University of Naples Federico II directly. They might be able to provide you with the contact information of the person in charge for the Autonomous Vehicle Engineering course or guide you to the right person.

Department of Engineering, University of Naples Federico II
Via Claudio 21, 80125 Napoli, Italy
Phone: +39 081 253 9111
Email: [ingegneria@unina.it](mailto:ingegneria@unina.it)

2. **Reach out to the Faculty of Engineering**: Another option is to contact the Faculty of Engineering at the University of Naples Federico II. They might be able to provide yo

### Contextual Compression: A Technique for Enhanced Retrieval

**NOTE: This method is not used in the final implementation.**

In this implementation, we utilize contextual compression as a technique to condense the context surrounding a query into a more compact representation. This compressed context is then used to retrieve relevant information from our vector database.

By passing the compressed context to our retrieval model, we enable it to pinpoint the most pertinent documents or vectors with greater accuracy. This approach has the added benefit of reducing the search space, allowing us to focus on the most promising candidates and ultimately improving the overall retrieval process.


In [73]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(chat_model)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [75]:
results = compression_retriever.invoke(
    "Who is the person in charge for autonomous vehicle engineering course?")
result[0]

Document(page_content='The course with code P38, named "INGEGNERIA DELL\'AUTOMAZIONE E ROBOTICA" (English name: "AUTOMATION ENGINEERING AND ROBOTICS"), is coordinated by DE TOMMASI GIANMARIA. It is part of the Laurea Magistrale program in the DIPARTIMENTO DI INGEGNERIA ELETTRICA E TECNOLOGIE DELL\'INFORMAZIONE department under the SCUOLA POLITECNICA E DELLE SCIENZE DI BASE school. The course is conducted in Italian and is located in NAPOLI. This course was activated in the year 2019. For more information, visit the course homepage: https://www.corsi.unina.it/P38.', metadata={'Code': 'P38', 'Coordinator': 'DE TOMMASI GIANMARIA', 'English name': 'AUTOMATION ENGINEERING AND ROBOTICS', 'Name': "INGEGNERIA DELL'AUTOMAZIONE E ROBOTICA"})

## Crafting Effective Prompt Templates for LLM Models

**NOTE:** Below text was generated by LLaMA.

To elicit the best possible responses from our LLM models, we'll follow these guidelines to create high-quality prompt templates.

* Ensure that your prompts are concise and easy to understand, avoiding technical jargon and using natural language instead.
* To provide additional context, ask the LLM to role-play as a character with a specific goal. For example, "You are a marketing executive who wants to invite major customers to a live event."
* Refine your prompt templates by iterating on different approaches to achieve the same goal. Seek feedback from end-users and analyze how well your prompts generate desired responses.
* Consistency is key: choose a writing style and stick to it, using consistent word choice, intensifiers, emojis, and punctuation to shape the LLM's
response.
* To help the LLM differentiate between context and instructions, create an "Instructions" section with triple quotes (""") in your prompt template.
This will guide the model to focus on generating specific content rather than discussing the process.
* Include direct instructions for the LLM to generate only the expected type of content. For example, if you want a draft email, specify that the
response should strictly follow these guidelines.

To get started, draw inspiration from the Example Prompt Template Library and customize templates to fit your unique needs. Study the language used
in the examples, paying particular attention to writing style and phrasing. You can apply similar techniques to create effective prompt templates for
your LLM models.


In [72]:
from langchain.prompts import SystemMessagePromptTemplate, PromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, ChatPromptTemplate


qa_system_prompt_template = """
As Frederick, your mission is to assist students in finding relevant information about courses, degrees, services, and general university details. To
answer questions accurately, please rely on the following guidelines.

* When answering, use the provided context to provide the most accurate response possible.
* If you're unsure or don't know the answer based on the available information, simply state "I don't know" instead of attempting to fabricate an
answer.

Additionally, follow these specific rules:
1. Link-only responses: If a question requests links, only return the source links without providing an answer.
2. Unknown answers: When you can't find the final answer based on available information, provide a clear indication by saying "I can't find the
final answer but you may want to check the following links" and include the relevant source links as a list.
3. Answer and sources: If you do find the answer, present it in a concise manner and accompany it with a list of sources that were directly used
to derive the answer. Exclude any sources that are not directly related to the final answer.

{context}"""

qa_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=qa_system_prompt_template
    )
)

qa_human_prompt_template = """Question: {input}
Answer:"""

qa_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template=qa_human_prompt_template)
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        qa_system_prompt,
        MessagesPlaceholder("chat_history"),
        qa_human_prompt,
    ]
)

### Crafting Contextualized Prompt Templates

When formulating prompt templates for our chatbot, we'll consider two key factors: the chat history and the latest user question. This context might
be referenced in the chat history, but we need to create a standalone question that can be understood without it.

Our goal is not to answer the question, but rather to reformulate it if necessary, or return it as is if it's already suitable for our chatbot.


In [76]:

contextualize_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

qa_system_prompt =  SystemMessagePromptTemplate.from_template(
    contextualize_system_prompt
)

qa_human_prompt_template = """Question: {input}
Answer:"""

qa_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template=qa_human_prompt_template)
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        qa_system_prompt,
        MessagesPlaceholder("chat_history"),
        qa_human_prompt,
    ]
)


### Establishing a Chain of Thought

#### Similarity-Based Search

In the realm of data retrieval from vector databases, the "similarity" search type focuses on identifying documents or vectors that closely match a given query. This approach relies on mathematical metrics like cosine similarity or Euclidean distance to calculate the similarity between the query vector and those stored in the database. Documents with higher similarity scores are deemed more relevant and are returned as search results. The "similarity" search type is commonly employed for tasks such as document retrieval, recommendation systems, and information retrieval.

#### Similarity Score Threshold

The "similarity_score_threshold" search type refines the basic similarity search method by introducing a threshold value to filter out search results based on their similarity scores. Only documents or vectors with similarity scores exceeding the specified threshold are returned as search results.
By setting an appropriate threshold, users can control the level of relevance and precision in the retrieved results. This search type is particularly useful in scenarios where a certain level of similarity is required for result inclusion, such as in content recommendation systems or search engines.

#### Maximal Marginal Relevance (MMR)

Maximal Marginal Relevance (MMR) is a search type that prioritizes diversifying search results by balancing relevance and diversity. Unlike
traditional similarity-based approaches that focus solely on high-similarity scores, MMR considers both the relevance of a document to the query and
its dissimilarity to already retrieved documents. This approach aims to provide a diverse set of search results that cover a wide range of relevant
topics or perspectives, making it particularly useful in tasks like information retrieval, summarization, and recommendation systems where diversity
in results is desired.


In [78]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.history_aware_retriever import create_history_aware_retriever
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.chains import (create_retrieval_chain)
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

retriever = vectordb.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

history_aware_retriever = create_history_aware_retriever(chat_model, retriever, contextualize_q_prompt)
question_answer_chain = create_stuff_documents_chain(chat_model, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Store chat history to manage session history
store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if user_id not in store:
        store[user_id] = ChatMessageHistory()
    return store[user_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)


def execute_conversation(input_text: str, session_id: str = "default"):
    var = conversational_rag_chain.invoke(
        {"input":input_text},
        config={"configurable": {"user_id": session_id, "conversation_id": "0"}},
    )
    return var.get("answer")

In [79]:
#@title Real-time chat history for each user
store

{}

## Testing the model.



In [80]:
#@title Let's try the model
print(execute_conversation("Can you provide the link for bachelor's degree of computer engineering?"))

Based on the provided context, I can provide the link for the course with code N46, which is the "INGEGNERIA INFORMATICA" (English name: "COMPUTER ENGINEERING") course. The link is:

https://www.corsi.unina.it/N46


In [81]:
print(execute_conversation("Who should I contact for the master's degree of architecture?"))

Based on the provided context, for the master's degree of architecture, you can contact PANE ANDREA, who is the coordinator of the course with code P53, named "ARCHITECTURE AND HERITAGE".


In [84]:
print(execute_conversation("Can you give me the website?"))

According to the provided context, you can visit the course homepage for more information:

https://www.corsi.unina.it/P53


# Gradio

Gradio is a user-friendly tool that enables you to create a web-based interface for your machine learning model. By defining the input and output types of your model, Gradio will automatically generate an interactive web interface that allows users to engage with your model. You can customize the interface by selecting from various input and output types, as well as styling options, to suit your specific needs.

Gradio also provides a shareable link to your interface, making it simple to share your model with others.


In [85]:
from uuid import uuid4
import gradio as gr


# Generate a session id for each load of webpage
# to keep the history of conversation for each user
def store_message(message: str, history):
    if not history:
        uuid = str(uuid4())
        history = [uuid]
    output = execute_conversation(message, history[-1])
    return output, history

demo = gr.Interface(
    fn=store_message,
    inputs=["text", gr.State(value=[])],
    outputs=["text", gr.State()]
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://79deb956a5cd7503ed.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [111]:
# %%writefile app.py
# import streamlit as st
# from uuid import uuid4
# import random

# # Function to simulate conversation execution
# def execute_conversation(message: str, session_id: str):
#     # Example responses based on the input message
#     responses = {
#         "hello": ["Hello!", "Hi there!", "Hey! How can I help?"],
#         "how are you": ["I'm good, thanks for asking!", "I'm doing well, how about you?"],
#         "bye": ["Goodbye!", "See you later!", "Have a great day!"]
#     }

#     if message.lower() in responses:
#         return random.choice(responses[message.lower()])
#     else:
#         return "I'm sorry, I don't understand that."

# # Function to store message and handle session history
# def store_message(message: str, history):
#     if not history:
#         session_id = str(uuid4())
#         history = [session_id]
#     else:
#         session_id = history[0]

#     output = execute_conversation(message, session_id)
#     history.append({"user": message, "bot": output})

#     return history

# # Streamlit app
# st.title("Conversation App")

# # Session state for storing history
# if 'history' not in st.session_state:
#     st.session_state.history = []

# # Text input for user message
# user_message = st.text_input("Enter your message:")

# # Button to send message
# if st.button("Send"):
#     if user_message:
#         st.session_state.history = store_message(user_message, st.session_state.history)
#         # Clear input after sending
#         user_message = ""

# # Display conversation history
# st.write("Conversation History:")
# for msg in st.session_state.history[1:]:
#     st.write(f"**You:** {msg['user']}")
#     st.write(f"**Bot:** {msg['bot']}")


Overwriting app.py


In [112]:
# import subprocess

# # Kill previous tunnels if any
# !pkill -f streamlit

# # Get the key
# NGROK_AUTHTOKEN = get_key("NGROK_KEY")

# # Function to kill any running ngrok tunnels
# def kill_existing_tunnels():
#     tunnels = ngrok.get_tunnels()
#     for tunnel in tunnels:
#         ngrok.disconnect(tunnel.public_url)

# # Kill previous tunnels if any
# kill_existing_tunnels()

# # Run Streamlit app in the background
# process = subprocess.Popen(['streamlit', 'run', 'app.py'])

# # Create a tunnel to the Streamlit app
# public_url = ngrok.connect(8501)
# print(f'Public URL: {public_url}')


Public URL: NgrokTunnel: "https://09a7-34-23-75-192.ngrok-free.app" -> "http://localhost:8501"
